In [ ]:
%pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
spark.conf.set(
    "fs.azure.account.key.devdolphinstorage.dfs.core.windows.net", 
    "key"
)


In [ ]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
import io
import os
import pandas as pd

In [ ]:
service_account_file = ""

In [ ]:
# Step 1: Setup GDrive connection (already shared)
creds = service_account.Credentials.from_service_account_file(
        service_account_file,
        scopes=["https://www.googleapis.com/auth/drive"]
    )
service = build('drive', 'v3', credentials=creds)

# Step 2: Define your folder and filenames
folder_id = "1qryhdlgNsmecWRy2haI8S3uC63wKk5X-"
transactions_file = "transactions.csv"
customer_file = "CustomerImportance.csv"

# Step 3: Download both files
def read_csv_from_gdrive(service, file_id):
    request = service.files().get_media(fileId=file_id)
    file_buffer = io.BytesIO()
    downloader = MediaIoBaseDownload(file_buffer, request)
    done = False
    while not done:
        _, done = downloader.next_chunk()
    file_buffer.seek(0)
    pandas_df = pd.read_csv(file_buffer)
    return spark.createDataFrame(pandas_df)

# Get file IDs
transactions_id = get_file_id(service, folder_id, transactions_file)
customer_id = get_file_id(service, folder_id, customer_file)

# Read as PySpark DataFrames
transactions_df = read_csv_from_gdrive(service, transactions_id)
customer_df = read_csv_from_gdrive(service, customer_id)